In [1]:
import sys 
import os 
sys.path.insert(0, os.path.dirname(os.getcwd()))

In [2]:
from utils import get_connection
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
con = get_connection()

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/


In [11]:
pd.read_sql("show databases ",con)


,Database
0,douyin
1,information_schema
2,mysql
3,performance_schema
4,validation
5,xiaohongshu


In [12]:
df_selective_mapping = pd.read_sql("SELECT * FROM validation.tag_group_text_topic_validation" ,con)
df_selective_mapping = df_selective_mapping.rename(columns= {'tag_old': 'tag_value', 'tag_new':'class_name'})

In [13]:
dict_sql = """
select tag_category, tag_value, keyword
from (
select * from validation.tag_kwd_text_topic
) a 
"""


In [14]:

df_dictionary = pd.read_sql(dict_sql, con = con)
print(df_dictionary.shape)
df_dictionary = df_dictionary.merge(df_selective_mapping, on = 'tag_value')
df_dictionary['kw_len'] = df_dictionary['keyword'].apply(len)
df_dictionary = df_dictionary.query("kw_len <= 8 & kw_len > 1")
print("filtered dictionary:" , df_dictionary.shape)

df_dictionary = (
    df_dictionary
    .groupby(['tag_category', 'class_name'])
    .apply(lambda x: list(set(x['keyword'])))
    .rename(index = 'kw_ls')
    .reset_index()
)

print("ttl kw:", df_dictionary['kw_ls'].str.len().sum())

df_dictionary.head()

(5817, 3)
filtered dictionary: (5376, 5)
ttl kw: 4349


,tag_category,class_name,kw_ls
0,文本主题,体重管理,"[减重, 生活化减肥, 体测, 晚餐打卡, 在校减肥, 减肥第二天, 放纵餐, 低卡甜品, ..."
1,文本主题,健康与养生,"[保护心血管, 健脾养胃, 艾草汗蒸, 医师资格考试, 改善睡眠, 熬夜, 脾胃虚弱, 清新..."
2,文本主题,健康与饮食,"[我爱eating, 取悦生活, 代餐, 生活化减肥, 喜欢吃糖, 益母草, 一人食吃什么,..."
3,文本主题,儿童饮食与健康,"[孩子吵闹时, 宝宝辅食, 宝宝, 脾胃虚弱, 宝宝早餐, 育儿, 宝宝的第一口米粉, 宝宝..."
4,文本主题,其他,"[哈密瓜, 好吃, 知识薯, 姥姥家, 显白穿搭, 性格温顺可爱软糯, 薯片, bm穿搭, ..."


In [9]:
def _exact_matching(x, kw_ls, tag_name):
    tag:str = None
    kw_matched = []
    for kw in kw_ls:
        if kw in x:
            tag = tag_name
            kw_matched.append(kw)
            # break
    return tag,kw_matched
    
def tag_data_one_category(content, df_dictionary:pd.DataFrame, kw_ls_col = 'kw_ls', tag_value_col = 'tag_value'):
    tag_value_ls = []

    for tag_value, _dictionary in df_dictionary.groupby(tag_value_col):
        kw_ls = _dictionary[kw_ls_col].values[0]

        tag, kw_matched = _exact_matching(content, kw_ls, tag_value)

        if tag:
            tag_value_ls.append({tag: kw_matched})
    return tag_value_ls
        
def tag_data_multi_category(data, df_dictionary, categories, category_col = 'tag_category', content_col = 'content', kw_ls_col = 'kw_ls', tag_value_col = 'tag_value'):
    content = data[content_col]
    rst = []
    for cate in categories:
        _df_dictionary = df_dictionary[df_dictionary[category_col] == cate]
        tags = tag_data_one_category(content, _df_dictionary, kw_ls_col, tag_value_col)
        rst.append(tags)
    return rst

def tag_data(df_dictionary, data,category_col = 'tag_category', content_col = 'content', kw_ls_col = 'kw_ls', tag_value_col = 'tag_value'):
    data = data.copy()
    categories = df_dictionary['tag_category'].unique().tolist()
    args = dict(category_col =category_col, content_col = content_col, kw_ls_col = kw_ls_col, tag_value_col = tag_value_col)
    data[categories] = data.progress_apply(tag_data_multi_category, axis = 1, result_type ='expand',df_dictionary = df_dictionary, categories= categories, **args)
    return data

In [10]:
from tqdm import tqdm 
tqdm.pandas()

In [226]:
data = pd.read_csv('data/validation_3000.csv')
data.head()

,id,content,output,text_topic
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐


In [227]:
tagged_data = tag_data(df_dictionary, data, tag_value_col='class_name')
tagged_data


  0%|          | 0/3000 [00:00<?, ?it/s]

100%|██████████| 3000/3000 [00:04<00:00, 696.64it/s]


,id,content,output,text_topic,文本主题
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战,"[{'其他': ['早起', '一拖二', '生活', '生活的人', '摆摊']}, {'..."
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐,"[{'体重管理': ['零食']}, {'健康与养生': ['零食']}, {'其他': [..."
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐,"[{'体重管理': ['奶茶']}, {'健康与饮食': ['夏天', '咸柠茶', '柠檬..."
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验,"[{'体重管理': ['零食']}, {'健康与养生': ['零食']}, {'其他': [..."
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐,"[{'其他': ['推荐', '儿童']}, {'广告与推广': ['福利', '推荐', ..."
...,...,...,...,...,...
2995,594ac51b1657f4738586e8ae24552b52|2be16c8d4707b...,#好吃又便宜的网红零食推荐 发现一款超好吃的小零食，就是这款九块九三十包的酥牛排，还是盐津铺...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '零食推荐...",零食推荐,"[{'体重管理': ['零食']}, {'健康与养生': ['零食']}, {'健康与饮食'..."
2996,357b96bad7f1fa478a787ae85785d175|6298eda597a46...,救命这个燕麦酥真的不要太好吃！\n这款燕麦巧克力酥不含反式脂肪，外皮包裹巧克力\n再配上饱满...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '零食推荐',...",零食推荐,"[{'体重管理': ['零食']}, {'健康与养生': ['零食']}, {'健康与饮食'..."
2997,956e86edcfd1f8201401df05c29181f3|846599ba7abdd...,尊敬的顾客朋友们\n🔔加一加水果零食明天4.18举行年会\n🙋🏻‍♀️全体员工参加⚠️闭店一...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '商业通知',...",商业通知,"[{'体重管理': ['零食']}, {'健康与养生': ['零食']}, {'其他': [..."
2998,7a6c58420d0aab1b5fc6f51a583503fe|9451fadf499ed...,#女大学生 #淄博烧烤 去淄博喽，八大局去了，紫米饼没吃，炒锅饼没吃，牛奶棒没吃，小蛋糕没吃...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅行体验',...",旅行体验,"[{'其他': ['朋友', '好吃', '女大学生']}, {'学习与教育': ['大学生..."


In [228]:
tagged_data['文本主题Max'] = \
    tagged_data['文本主题'].apply(
        lambda x: [list(i.keys())[0] for i in x][np.array([len(i[list(i.keys())[0]]) for i in x]).argmax()] if len(x) else x
    )


In [229]:
tagged_data.to_csv('data/3000dictionary_on_3000.csv', index=False)

In [197]:
list(x[0].keys())[0]

'体重管理'

In [153]:
tagged_data['文本主题'].apply(lambda x: pd.Series(x.split(',')).value_counts().to_dict())

0       {'体重管理': 1, '健康与养生': 1, '健康与饮食': 1, '其他': 1, '...
1       {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
2       {'体重管理': 1, '健康与养生': 1, '健康与饮食': 1, '儿童饮食与健康':...
3       {'体重管理': 1, '健康与养生': 1, '运动健身': 1, '节日庆祝': 1, ...
4       {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
                              ...                        
2995    {'体重管理': 1, '健康与养生': 1, '运动健身': 1, '节日庆祝': 1, ...
2996    {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
2997    {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
2998    {'体重管理': 1, '日常记录': 1, '运动健身': 1, '节日庆祝': 1, '...
2999    {'体重管理': 1, '旅行': 1, '运动健身': 1, '节日庆祝': 1, '美食...
Name: 文本主题, Length: 3000, dtype: object

In [115]:
pd.read_sql('show tables', get_connection(database= 'validation'))

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/validation


,Tables_in_validation
0,tag_group_text_topic
1,tag_group_text_topic_validation
2,tag_kwd_character_type
3,tag_kwd_emotional_atmosphere
4,tag_kwd_food_category
5,tag_kwd_food_usage
6,tag_kwd_is_retail_food
7,tag_kwd_macro_occasion
8,tag_kwd_micro_occasion
9,tag_kwd_painpoint


In [125]:
from io import StringIO
import json

data = pd.read_csv(StringIO(text), sep= '\t')

data['output'] = data['output'].apply(lambda x: json.loads(x.replace("'", '"')))

In [127]:
data['text_topic'] = data['output'].apply(lambda x: x['文本主题'])

In [132]:
data.to_csv('data/validation_3000.csv', index=False)

In [131]:
data

,id,content,output,text_topic
0,e04592c0da33650d9c837464dca6c706|b9393c69e2e6d...,一拖二的周六，一早起床，勿勿忙忙准备好小娃需要看病的资料，再准备我需要摆摊的工具，大娃补课催...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '日常生活挑战...",日常生活挑战
1,18336dc645f26e3c3944e8c47c1c9831|d035d1c7e67d6...,津铺子的这个0卡果冻回购很多次了，一共四种口味：白桃、青提、荔枝和芒果。四个口味都还不错，口...,"{'食品类别': '蜜饯果干', '是否为零售食品': '是', '文本主题': '产品推荐...",产品推荐
2,b9240d65115b682962187bac31a5a14d|e64571aeb3fbf...,话梅柠檬茶！建议女孩子都把奶茶换成它！ 全网安利的咸柠茶！我能喝一整个夏天！真的好喝到原地转...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '饮品推荐',...",饮品推荐
3,a0e6ebe573ea14303727123a5abbef3e|de754b5d37397...,随手买的香辣鱼尾也太好吃了吧！鱼肉很有嚼劲，麻麻辣辣的也太过瘾了，平时追剧或者嘴馋了来一包，...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '食品体验...",食品体验
4,d3fc3728a8c3e13ae5988aa15e4bb0fd|320b8f65d7782...,#踏春赏花最美的景点推荐 #感受大自然的气息和美景 #旅行推荐官 #抖音带你去赏花 #春暖花...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅游推荐',...",旅游推荐
...,...,...,...,...
2995,594ac51b1657f4738586e8ae24552b52|2be16c8d4707b...,#好吃又便宜的网红零食推荐 发现一款超好吃的小零食，就是这款九块九三十包的酥牛排，还是盐津铺...,"{'食品类别': '肉脯卤味', '是否为零售食品': '是', '文本主题': '零食推荐...",零食推荐
2996,357b96bad7f1fa478a787ae85785d175|6298eda597a46...,救命这个燕麦酥真的不要太好吃！\n这款燕麦巧克力酥不含反式脂肪，外皮包裹巧克力\n再配上饱满...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '零食推荐',...",零食推荐
2997,956e86edcfd1f8201401df05c29181f3|846599ba7abdd...,尊敬的顾客朋友们\n🔔加一加水果零食明天4.18举行年会\n🙋🏻‍♀️全体员工参加⚠️闭店一...,"{'食品类别': '未知', '是否为零售食品': '是', '文本主题': '商业通知',...",商业通知
2998,7a6c58420d0aab1b5fc6f51a583503fe|9451fadf499ed...,#女大学生 #淄博烧烤 去淄博喽，八大局去了，紫米饼没吃，炒锅饼没吃，牛奶棒没吃，小蛋糕没吃...,"{'食品类别': '未知', '是否为零售食品': '否', '文本主题': '旅行体验',...",旅行体验


In [86]:
con = get_connection()
n = pd.read_sql(f"select count(*) count from ({data_sql}) a", con = con)['count'].values[0]
chunksize = 100
step = int(n/chunksize)
n
for data in tqdm(pd.read_sql(data_sql, con = con, chunksize = chunksize), total = step):
    data = tag_data(df_dictionary, data)
    data[['id', 'ds', *df_dictionary['tag_category'].unique().tolist() ]].to_csv('data/tagged_validation.csv', mode = 'a', index=False)

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/


  0%|          | 0/28 [00:00<?, ?it/s]


KeyError: 'kw_ls'

In [11]:
import multiprocessing as mp
from multiprocessing import Pool
con = get_connection(database='xiaohongshu')
n_process = mp.cpu_count()

def func(data):
    return tag_data(df_dictionary, data)
    
n = pd.read_sql(f"select count(*) count from ({data_sql}) a", con = con)['count'].values[0]
chunksize = 10
step = int(n/chunksize)
iterator = pd.read_sql(data_sql, con = con, chunksize = chunksize)

if __name__  == '__main__':
    pool = Pool(n_process)
    for d in tqdm(pool.imap_unordered(func, iterator), total = step):
        d[['id', 'ds', *df_dictionary['tag_category'].unique().tolist() ]].to_csv('data/tagged.csv', mode = 'a', index=False)

connecting to rm-uf6lfxk83h7onjmwvjo.mysql.rds.aliyuncs.com3306/xiaohongshu


  0%|          | 0/587164 [00:00<?, ?it/s]

In [28]:
data.loc[:1,'content'].values[0]

'先说万豪白金不烂大街哦[萌萌哒R]北上广江浙一带商旅人士多，三亚旅游的人多，可能大家看到的现象就觉得万豪白金烂大街，二三线城市还是非常非常好用的，具体权益和获取办法可以看我前面笔记[暗中观察R]\n\t\n[举手R]今天给大家讲一个小众且高级的酒店联盟：GHA\n大名鼎鼎的尼依格罗、三亚嘉佩乐就是这个联盟。\n划重点：几百块钱能升级的[二R]年有效期酒店🔝会籍，只有GHA钛金！（之前bug渠道已修复）\n\t\n[举手R]GHA钛金会员权益：\n1：客房连升两级，包括行政房或套房，房型差价1k+\n2：大部分酒店视情况供应免费双早和行政待遇，需要提前问酒店\n3：提前至11点入住，次日可延迟至16点退房。\n4：欢迎水果和点心\n5：免费游泳池健身房\n\t\n而且它秒升！秒升诶！[赞R]着急入住的快冲[赞R]'